In [ ]:
Total: 80

<b>Jim Fitzgerald - 819812741 - CS696 Assignment 3

<hr><b>1 Write a function that will put N doubles into a file. The doubles need to be normally dis-
tributed with mean 0 and standard deviation 1. The function should have two arguments: N
and the full name of the file (ie includes path to file location).</b>

In [1]:
def gaussian(N: Integer, filename: String) = {
    val rdd_random = sc.parallelize(Array.fill(N){scala.util.Random.nextGaussian()})
    rdd_random.saveAsTextFile(filename)
}


In [2]:
val file = "jf.csv"
gaussian(50000, file)

<hr><b>2 Read the file created in #2 into an RDD and compute the mean and standard deviation of
the doubles in the file. Work on the RDD, that is do not convert the RDD to a DataFrame or
Dataset.. You are to use Spark code to compute the values as we want this to run on a
cluster using multiple machines. So the pure Scala code you used in assignment will not
work.</b>

In [3]:
val rdd = sc.textFile(file)
val asDouble = rdd.map(x => x.toDouble)
asDouble.mean()

0.007114136923324765

In [4]:
asDouble.stdev()

0.9991214012552679

In [5]:
asDouble.stats()

(count: 50000, mean: 0.007114, stdev: 0.999121, max: 4.220758, min: -4.229699)

<hr><b>3 Repeat #3 but using a DataFrame instead of RDD. Here work on the DataFrame not an
RDD.</b>

In [6]:
import org.apache.spark.sql.functions._ 
val reader = spark.read
reader.option("header", false)
reader.option("inferSchema", true)
val df = reader.csv(file)

In [7]:
val allData = df.selectExpr("mean(_c0) as Mean", "stddev_samp(_c0) as Samp", "stddev_pop(_c0) as Pop")
allData.show()

+--------------------+------------------+-----------------+
|                Mean|              Samp|              Pop|
+--------------------+------------------+-----------------+
|0.007114136923324753|0.9991313926191513|0.999121401255268|
+--------------------+------------------+-----------------+



</hr><b>4 Using a DataFrame create a random sample of about 100 elements of the file created in #2
and compute the mean of the sample.</b>

In [8]:
val s1: Double = 100.0/50000.0  // 0.002 yields 85 results, need 0.0023 for 100
val s2: Double = (50000-100)/50000.0
val sampling = df.randomSplit(Array(0.0023, 0.998), 5)
println(sampling(0).count())    // just to show 100 results
val allData4 = df.selectExpr("mean(_c0) as Mean", "stddev_samp(_c0) as Samp", "stddev_pop(_c0) as Pop")
allData4.show

100
+--------------------+------------------+-----------------+                     
|                Mean|              Samp|              Pop|
+--------------------+------------------+-----------------+
|0.007114136923324753|0.9991313926191513|0.999121401255268|
+--------------------+------------------+-----------------+



</hr><b>5 Create a file of 100 normally distributed doubles. Read the doubles from the file into an
RDD. Using the RDD create a sliding window of size 20 and compute the mean of each
window.</b>

In [9]:
val file5 = "file5.txt"
gaussian(100, file5)

In [10]:
import org.apache.spark.mllib.rdd.RDDFunctions._

val rdd5 = sc.textFile(file)
val asDouble5 = rdd5.map(x => x.toDouble)
val slide = asDouble5.sliding(20)
val d2 = slide.map(x=>x.sum/x.size).collect()
// only way I could find to print the RDD Array[Double]
d2.foreach(println)

-0.08818461603236427
-0.012049127044321853
-0.04052222343069972
-0.07795363118122642
0.0029227656767888076
-0.026490280989235337
-0.0016711646151612819
-0.030603040341127458
-0.04366253703826082
-0.10060819444539522
-0.06703899638845069
-0.0900606006375098
-0.21192482377085214
-0.22213715723389588
-0.3552771415995689
-0.359641762432559
-0.2832067251711934
-0.31337075804101655
-0.3636221238252072
-0.3483102527077283
-0.2801364805089366
-0.258122699472987
-0.22053281096179034
-0.20363008626622853
-0.2708737658603209
-0.15137154047306253
-0.20747709448568466
-0.17575216082228903
-0.2199062354820906
-0.20262130512308904
-0.18257928957996133
-0.14508284564360158
-0.04013887466003341
-0.07453392758809807
0.06710319135711947
0.09285152702985032
0.14923739378848339
0.18298053026541122
0.3436565953615314
0.36528385551268583
0.2188373294593319
0.2179085860830136
0.2042935987770468
0.19682335883396349
0.26132031443966336
0.28939484340803234
0.31689861143972686
0.28274659138808866
0.21152658112979

-0.21551001111781104
-0.19704385272036526
-0.30633672957089797
-0.2458267123457183
-0.40398173563102235
-0.43411584516103163
-0.5531927410187281
-0.6660700614585151
-0.6945516028471752
-0.6799224031615432
-0.5618567047287286
-0.6179795694465977
-0.616816005162694
-0.5205492513809131
-0.4653343345201824
-0.4840395465396516
-0.32430547761377
-0.271534792680468
-0.2122865513332582
-0.2782765726227358
-0.20534917233036137
-0.17401863457906092
0.008212435942823204
0.03526677087781891
0.07036589135535976
0.15828859379010463
0.29684195135452507
0.39001923464841287
0.38171618054006723
0.3623448682375153
0.4169506239252957
0.4798733283378785
0.5056585488739924
0.5398088332987744
0.4516259549876403
0.3588358961139203
0.32229423791638123
0.25460710491158967
0.27355075377006843
0.3784856969667995
0.3464983290652246
0.3360076007058924
0.26092040484435786
0.2951478878309379
0.3661046376746723
0.4739614168905894
0.3223359406845342
0.34356084723875013
0.42001180543812433
0.3729850022554881
0.230046701

0.09796133402747649
0.09690675448244045
0.1651384056246779
0.10763872139021965
0.17703469641699277
0.1679537998110526
-0.012740652231891636
-0.027565386305321092
-0.04405006020747856
-0.18101577762503754
-0.1766586930934249
-0.13410534489182407
0.002461298702866399
0.02297886028796877
-0.002322781788111189
-2.3155365935178252E-4
-0.07272191685170312
-0.16500570744399262
-0.1713071892466806
-0.2367359006013528
-0.21822790866661368
-0.23857370759343183
-0.2484032747063961
-0.10914997621691817
-0.1543231997952435
-0.10546955476797346
-0.016670728122458282
-0.03161699154976398
-0.09999846109404192
-0.054372186209272
-0.05947188619386068
-0.14502832242515964
-0.3244759516262867
-0.3932000731596174
-0.4145785535146147
-0.5464044900653511
-0.5882971495477155
-0.6933882431689946
-0.663361854577916
-0.6353349057301553
-0.623061778254934
-0.5380440572974423
-0.5685161997675598
-0.5820340280204004
-0.5284450754442329
-0.5095299424788421
-0.49318249423236116
-0.45997954753798875
-0.361362664774434

0.32717823226012743
0.37790854009435526
0.39300702603376336
0.40863067805559866
0.5104256564461328
0.46588560517696787
0.4577011159518989
0.3876760640015799
0.46414096600296756
0.35215106131473023
0.39050169945110597
0.4074947306811249
0.47633465673288156
0.356899325206235
0.424008254057233
0.47988197645568664
0.40660510424200336
0.3519289644664113
0.13116333668831337
0.23141983486214057
0.2504342982602976
0.23000495814732097
0.213854554785595
0.13666087570052401
0.06790345068216617
0.14188275469227177
0.16046698568775306
0.1695451631024151
0.16719688201559285
0.2677905139602964
0.3216002785033976
0.2846714814009234
0.14819439293406
0.24168444608325643
0.21319336334157807
0.12465030564666704
0.052692850960462
0.13846574985363153
0.22318505676736283
0.08415428074258649
0.05851517572110716
0.058514333471513205
0.018737053349196037
0.07512548327218216
0.16747711692228245
0.1088139373995837
0.0363795570847043
0.07702821215551708
0.09624729178591784
0.085900482250143
0.004458062159752307
-0

0.07770000333982705
0.09148418981199677
0.04178621269824942
0.032161679995191296
0.11249145388882482
0.061067675782807626
0.04819918520074483
0.05783142423938936
0.12924973077754198
0.03318106348512907
-0.015470608147634055
-0.07372403786093296
-0.14612229527944037
-0.21696179264290683
-0.14662667549947336
-0.09655192916539015
-0.06884408714051732
0.023987353165096245
-0.061829986807793946
-0.09198100057051095
-0.19556572928129934
-0.24832122302685908
-0.24591456761397104
-0.2101342318212192
-0.2396267003295413
-0.28317948998847015
-0.41856382297762684
-0.45834603050933975
-0.48034178935750826
-0.3985521939425708
-0.40805042915177464
-0.33822130425764024
-0.3650294818226557
-0.24245492420317544
-0.2640364530703913
-0.26650938206255964
-0.2601771705392454
-0.2751105635526191
-0.22444332551765406
-0.23176519296647805
-0.12254593299644734
-0.009450273622717465
0.053495036921950845
-0.003914369813891511
0.037438093473717585
0.019101152405923406
0.023664227306538187
-0.023384559125733718
-0

...

0.5254536884248064
0.4809859872530321
0.5512215838535555
0.584948222606325
0.5956402629089254
0.6433071556780233
0.6610031545630124
0.4666785239613661
0.431130429906949
0.37808313441824637
0.35967515047573106
0.3470773174171104
0.17774522134561996
0.2319862181789872
0.13881524234847448
0.17630395152722705
0.2350624994989626
0.26464584059459073
0.24986297890905007
0.27392878755061856
0.2639578868669282
0.31458702970188934
0.19995846780859886
0.17354269379491982
0.0687596880771173
0.0067222631813061125
0.0528933875989084
0.09371148557193623
0.1893357467682543
0.2697747117801398
0.27786965280782017
0.27832695173249
0.3474651003310797
0.3437249826088354
0.36313288606779154
0.2781443963123227
0.20636362949126727
0.18746600798537047
0.11739182577025191
0.06365925072532011
0.09167903795813186
0.029939400442789865
0.04972840606957063
0.06232098008177335
0.13559430681373427
0.19633617540031748
0.11050269969853702
0.09494102863796819
0.0633211764701212
0.008100473315826435
0.026996363161839825
0

0.10235695177312412
0.09020177222634448
-0.005739061717396393
0.0703006810321031
-0.027820961856577547
-0.0318756082119628
-0.06156137020675982
-0.015970825690598853
0.068277420441782
0.170683250904682
0.1716354466047835
0.20449960026365982
0.005569773564461844
0.018362564594562925
0.0713318006854133
-0.11175203772098614
-0.09241857657036714
-0.05273773752180605
-0.08111153942826357
-0.028655784933573035
0.07827072942953453
0.03244777130292236
0.027215288734003744
0.058622806715966515
0.08538746149723794
0.08461528967278006
0.09216800880175374
0.12096840670039463
0.11646085662795683
0.051866734994895226
0.13683901381578248
0.07973209680553131
0.20092977559919892
0.1436451368742678
0.0907574088421364
0.0922742027117775
0.09036120124421528
0.09010472089210911
0.14635851606827596
0.0228849262205131
0.06791212280183581
0.08122591791882182
0.18617142087035254
0.1663285311660183
0.12238408844239616
0.15626014093062027
0.17241260407000913
0.16448944790344527
0.24975015982891985
0.195621966294

0.5812675921685061
0.6265593027865866
0.6560144320814729
0.6567780592659276
0.6016037211276676
0.6784744168878187
0.6110821611575198
0.5645640855801946
0.45153309360188165
0.3562174941807349
0.1829492938205408
0.15083714284938496
0.1445731374959394
0.03164631053869983
0.10314497775988048
0.04201730593367829
-0.04802836775525613
0.01022226955386306
0.027806498604372575
-0.0035798911363951746
-0.10086385599375129
-0.12319342465745138
-0.2059388674880704
-0.298443211103441
-0.24570015039487436
-0.323464594711511
-0.2534112306916846
-0.2842990340536701
-0.14679919262777835
-0.16904521821218504
-0.09136111602117572
-0.009120566065024094
0.01931834691742228
-0.001184669084371412
-0.05334149917551362
-0.029070987253125825
0.04545188461113555
0.08430121017891369
-0.025915541923579255
-0.0840991157133096
-0.07166313904363866
-0.10090908023334329
-0.11158831972494672
-0.03619431658022344
-0.06944084324715542
-0.03146552998698026
-0.09635704303883837
0.011898764809646484
-0.022129416783194057
0.0

0.3188531079374996
0.1953860770839235
0.22519927862348493
0.26991581987680224
0.4040450215037839
0.41247838959510696
0.25140865458494066
0.18539859848508156
0.09756963809743216
0.15044803860746037
0.1464559296782965
0.013061294776510269
0.05665650585007982
0.15805851215741923
0.15665694750963383
0.13611487544633452
-6.18338082144465E-4
-0.05958039162621211
-0.11958843330262056
-0.17954473312002106
-0.12545456303952773
-0.1321882412498628
-0.10683550983081773
-0.09418773701658982
-0.19229420375628736
-0.18912881242567842
-0.09246787572192093
-0.09834963738374396
-0.12324646519385044
-0.14391543866317347
-0.06430836205598986
0.029581552910177188
0.004871012051793277
-0.031800052733582905
-0.07713870175154426
-0.019328681957494646
0.027261815169600533
0.0375049936410542
0.11445620776064853
0.08864932447365971
0.017365693139484956
-0.04613818104951854
-0.05818946968578611
-0.05816490941853778
-0.0057171741828676795
-0.01922229128296373
0.04897255015778294
0.14527227336137372
0.094443384171

-0.08395797470437738
-0.17685199805406868
-0.19744414322423398
-0.3427007302643332
-0.3951844701172648
-0.4123948263984533
-0.4155201324290122
-0.4449641579001981
-0.48741766717723073
-0.4378262163401912
-0.4364067808041465
-0.34170855726920635
-0.3399483021893138
-0.2517732030010184
-0.1589512617532254
-0.08363304241546439
-0.13907216287784868
-0.1535271163314999
-0.18191740456777045
-0.2128124161961086
-0.09307939528433687
-0.030979905210010907
-0.07052901684687232
-0.026229037652852023
-0.0024414957300494576
-0.04479187681411971
-0.08485835000461298
-0.10509460108890319
0.04448255956692241
0.021892753521184916
-8.858920211126664E-4
-0.08125137452932475
-0.13584752764910923
-0.1352135716447591
-0.11970777416264992
-0.17161290533133783
-0.16285350393841347
-0.06071742828318125
0.031044720046242036
-0.008535122477383139
0.01571222550184504
0.016513305768603236
0.1350582441323503
0.2802870296558487
0.29103511919924857
0.30528259522918194
0.250871708715103
0.30204825443592587
0.268229450

</hr><b>6 The file “multiple-sites.tsv” contains two columns: site and dwell-time. Using Spark compute
the average dwell time for each site.</b>

In [11]:
val file6 = "multiple-sites.tsv"
val reader = spark.read
reader.option("header", true)
reader.option("inferSchema", true)
reader.option("delimiter", "\t")
val df6 = reader.csv(file6)

In [12]:
val sorted = df6.groupBy("site").
    agg(
        mean("dwell-time").alias("Average on Site")).
    sort("site")
    
sorted.show


+----+------------------+
|site|   Average on Site|
+----+------------------+
|   0| 79.85106382978724|
|   1|             106.0|
|   2| 88.22916666666667|
|   3| 97.47916666666667|
|   4| 94.33333333333333|
|   5|102.33333333333333|
|   6|144.19298245614036|
|   7|123.36734693877551|
|   8| 94.34693877551021|
|   9|             89.82|
|  10|129.95238095238096|
|  11| 96.98214285714286|
|  12| 80.95081967213115|
|  13| 90.73770491803279|
|  14| 74.76470588235294|
|  15|119.34782608695652|
|  16| 86.74418604651163|
|  17|  77.8913043478261|
|  18| 94.81481481481481|
|  19| 89.28070175438596|
+----+------------------+



</hr><b>7 The file “multiple-sites.tsv” contains two columns: date and dwell-time. Using Spark com-
pute the following:
1. The average dwell time each hour
2. The average dwell time per day of week
3. The average dwell time on week-days (Monday - Friday)
4. Average dwell time on the weekend.</b>

In [13]:
val file7 = "dwell-times.tsv"
val reader = spark.read
reader.option("header", true)
reader.option("inferSchema", true)
reader.option("delimiter", "\t")
val df7 = reader.csv(file7)

In [14]:
//The average dwell time each hour
val times = df7.withColumnRenamed("date", "Time Stamp").
    withColumn("Date", to_date(col("Time Stamp"))).  // extract date to a new column
    withColumn("Hour", hour(col("Time Stamp"))).
    withColumn("Month", date_format(col("Date"),"MMMM")).
    withColumn("Day", date_format(col("Date"),"EEEE"))
val hourly = times.groupBy("Hour").
    agg(
        mean("dwell-time").alias("Average Time on Site by Hour")).
    sort("Hour")
hourly.show(24)

+----+----------------------------+
|Hour|Average Time on Site by Hour|
+----+----------------------------+
|   0|             94.708670095518|
|   1|           92.20954287620954|
|   2|           96.85937970490816|
|   3|           92.57110924839341|
|   4|           91.33086825527253|
|   5|           96.14621040723982|
|   6|           92.17173727608757|
|   7|           94.68557487038731|
|   8|           92.35551839464883|
|   9|           90.15113156885309|
|  10|           92.52882433045846|
|  11|           92.37820848611838|
|  12|           91.21507890122736|
|  13|           92.82834185536888|
|  14|           95.47408666100254|
|  15|           90.56982193064667|
|  16|            93.1575817641229|
|  17|           91.66231155778894|
|  18|           93.13833634719711|
|  19|            88.4258114374034|
|  20|           95.00888324873097|
|  21|           91.80379403794038|
|  22|           97.38431752178121|
|  23|           93.45957962813257|
+----+----------------------

In [15]:
//2 The average dwell time per day of week
val daily = times.groupBy("Day").
    agg(
        mean("dwell-time").alias("Average Time on Site by Day")).
    sort("Day")
daily.show

+---------+---------------------------+
|      Day|Average Time on Site by Day|
+---------+---------------------------+
|   Friday|          90.44266729389628|
|   Monday|          89.11023872679046|
| Saturday|         116.88253012048193|
|   Sunday|         106.49005681818181|
| Thursday|           91.4947995556902|
|  Tuesday|          89.73922367574522|
|Wednesday|          91.57283771155643|
+---------+---------------------------+



In [16]:
//3 The average dwell time on week-days (Monday - Friday)

val weekdays = times.filter(col("Day") =!= "Saturday").
    filter(col("Day") =!= "Sunday").
    agg(
        mean("dwell-time").alias("Average Time on Site Weekdays"))
weekdays.show

+-----------------------------+
|Average Time on Site Weekdays|
+-----------------------------+
|             90.4893776181927|
+-----------------------------+



In [17]:
// 4 The average dwell time on weekends
val weekends = times.filter(col("Day") === "Saturday" or col("Day") === "Sunday").
    agg(
        mean("dwell-time").alias("Time on Site Weekends"))
weekends.show

+---------------------+
|Time on Site Weekends|
+---------------------+
|   110.79575707154743|
+---------------------+



</hr><b>8. Do the average dwell times computed in #7 indicate any difference in users behavior?</b>

The first major indicator is the difference between Weekdays and Weekends.  Users spend more time on average on the site during weekends than they do on weekdays.  This indicates it is more likely to be a site for leisure than for business purposes.

Users spend the least amount of time on the site at around 7pm - perhaps don't use it during family time, dinner, homework.  They spend more time in the late hours - averaging 96/97 at 11 pm, 2 am, and 5am.  These do not seem to be regular times for a user to be spending so much time on the site.  

In fact, there isn't much of a change at all during regular sleeping hours.  The fault in this assumption is relying on US sleeping hours.  Or at least that the timestamps provided match the time zone of the users.  If this is a site frequented by users around the world, then it seems reasonable that the average dwell-time doesn't seem to drop for any work/sleep periods.